In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/MIDAS_hongxin

[Errno 2] No such file or directory: 'drive/My Drive/MIDAS_hongxin'
/content/drive/My Drive


In [0]:
ls

'Copy of Model.ipynb'   MidasExport.csv   new_df.csv   project_profit.csv


In [0]:
!ls

 calcs.docx					       Model.ipynb
 fields.xlsx					       new_df.csv
'MIDAS Data Challenge 2019_WalterPMoore_Prompt.docx'   Notes.gdoc
 MidasExport.csv				       project_profit.csv
 MIDAS.ipynb					       Report.gdoc


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import csv

In [0]:
df = pd.read_csv("project_profit.csv", encoding = "ISO-8859-1")

In [0]:
df.shape

(10126, 30)

In [0]:
df.head()

,ProjectFamily,NonIndiaHrsPED,LaborEffortPED,TotalHoursPED,PrincipalProjectCountPED,PMProjectCountPED,LaborHrsPED,EngUniqueCount,EngProjectCountPED,EngHoursPED,EmpCountPED,TypeCode,ProjectManager,ProjectEngineer,Principal,ParentClientTypeCode,ParentClientCode,OwningOrgCode,DivisionHead,ClientTypeCode,ClientState,ClientCode,ClientCity,HoursPTD,TotalBudgetHours,LaborCostMTD,LaborOverheadMTD,EffortMTD,BillingMTD,profit
0,1204001,31.50,2548.52,31.50,0,11,31.50,0,11,0.00,16,P06,121014,121014.0,121012,DE,UPTOWN,Z91,121012,DE,Texas,UPTOWN,Houston,0.0,0.0,910.19,0.000000,2603.65,2603.63,1
1,2300002,770.75,45814.18,770.75,1196,382,770.75,91,653,582.25,381,P06,230173,230173.0,230173,DE,LOUETT,T03,230009,DE,Texas,LOUETT,HOUSTON,0.0,55.0,9766.85,19748.315675,29553.13,56857.39,1
2,2300028,9.00,607.49,9.00,141,71,9.00,1,71,3.00,5,E02,230018,230018.0,230021,AR,KENN,C03,230009,AR,Missouri,KENN,ST. LOUIS,0.0,86.0,216.96,433.703040,607.49,6000.00,1
3,2300033,19.00,1173.18,19.00,420,53,19.00,15,314,13.00,39,P06,230195,230141.0,230021,AR,ZCA,C03,230009,AR,Texas,ZCA,Houston,10.0,0.0,603.34,1219.921078,1888.73,319.37,1
4,2300036,11.50,907.59,11.50,269,41,11.50,6,112,11.50,14,C06,230125,230018.0,230021,AR,J*R,C03,230009,AR,Texas,J*R,HOUSTON,2.5,70.0,302.53,573.544123,933.65,1211.30,1


In [0]:
#% used labor
df["per_used_laborMTD"] = df["LaborRevenueMTD"]/df["TotalBudgetEffort"]
df['per_used_laborPTD'] = df["LaborRevenuePTD"]/df['TotalBudgetEffort']

In [0]:
# df.groupby('ProjectFamily')['FamilyRevenueMTD'].max()
# pd.DataFrame(pd.DataFrame(df.groupby(['ProjectFamily','AccountingPeriod'])['profit_amtMTD'].first()).groupby('ProjectFamily')['profit_amtMTD'].sum())

In [0]:
tot_revenue = pd.DataFrame(pd.DataFrame(df.groupby(['ProjectFamily','AccountingPeriod'])['FamilyRevenueMTD'].first()).groupby('ProjectFamily')['FamilyRevenueMTD'].sum()).reset_index()
tot_revenue = tot_revenue.rename(columns = {'FamilyRevenueMTD':'Total_Revenue'})

In [0]:
tot_cost = pd.DataFrame(pd.DataFrame(df.groupby(['ProjectFamily','AccountingPeriod'])['FamilyCostMTD'].first()).groupby('ProjectFamily')['FamilyCostMTD'].sum()).reset_index()
tot_cost = tot_cost.rename(columns = {'FamilyCostMTD':'Total_Cost'})

In [0]:
tot_overhead = pd.DataFrame(pd.DataFrame(df.groupby(['ProjectFamily','AccountingPeriod'])['FamilyOverheadMTD'].first()).groupby('ProjectFamily')['FamilyOverheadMTD'].sum()).reset_index()
tot_overhead = tot_overhead.rename(columns = {'FamilyOverheadMTD':'Total_Overhead'})

In [0]:
proj_prof =pd.merge(tot_overhead,tot_revenue,on = 'ProjectFamily').merge(tot_cost,on = 'ProjectFamily')

In [0]:
proj_prof["profit_amt"] = proj_prof['Total_Revenue'] - (proj_prof["Total_Overhead"] + proj_prof['Total_Cost'])
proj_prof["profit_per"] = (proj_prof['Total_Revenue'] - (proj_prof["Total_Overhead"] + proj_prof['Total_Cost']))/proj_prof['Total_Revenue']


In [0]:
proj_prof['profit'] = proj_prof['profit_amt']
proj_prof['profit'] = proj_prof['profit'].apply(lambda x: 1 if x > 0 else 0)
proj_prof = proj_prof.rename(columns={"profit_amt":"total profit"})

In [0]:
proj_prof['Billing'] = pd.DataFrame(df.groupby(['ProjectFamily','AccountingPeriod'])['BillingMTD'].first()).groupby('ProjectFamily')['BillingMTD'].sum().values

In [0]:
proj_prof.head(10)

In [0]:
# #Profit amt
# df["profit_amtMTD"] = df["FamilyRevenueMTD"] - (df["FamilyCostMTD"]+df["FamilyOverheadMTD"])
# df["profit_amtPTD"] = df["FamilyRevenuePTD"] - (df["FamilyCostPTD"]+df["FamilyOverheadPTD"])

In [0]:
# #Profit %
# df["profit_perMTD"] = (df["FamilyRevenueMTD"] - (df["FamilyCostMTD"]+df["FamilyOverheadMTD"]))/df["FamilyRevenueMTD"]
# df["profit_perPTD"] = (df["FamilyRevenuePTD"] - (df["FamilyCostPTD"]+df["FamilyOverheadPTD"]))/df["FamilyRevenuePTD"]


In [0]:
#WIP
df["wip"] = df["FamilyRevenuePTD"] - df["BillingPTD"]

In [0]:
temp = df.loc[df['ProjectFamily'] == "2300002",:]

In [0]:
temp.sort_values(by = ['ProjectCode',"PeriodEndDate"]).loc[:,['PeriodEndDate','ProjectCode','CostMTD','CostPTD','FamilyCostMTD','FamilyCostPTD']].head(100)
#Observe that the cost of family does not always increase with time. This might be some error when entering the data. Meanwhile, for different sub-projects, the familycost does not cumulate the cost, but to start again every new
#subproject. So, for each subproject, I will the sum of the MTD data to represent the PTD.

In [0]:
#create a new dataframe to store the profitablility of a project family
# proj_prof = pd.DataFrame(pd.DataFrame(df.groupby(['ProjectFamily','AccountingPeriod'])['profit_amtMTD'].first()).groupby('ProjectFamily')['profit_amtMTD'].sum())

In [0]:
#Some visualization


In [0]:
#Make a new dataframe, for each project family, keep one record in it. The record is the cumulation of the information in the original dataframe, such as the MTDs. The response is the profit and the rest will be predictors.
#The variable that are same for same project family:
#The variables that need to be added for one proect family: BillingMTD, CostMTD, EffortMTD,  

In [0]:
df['IndiaCountPED'].value_counts().max() > df['IndiaCountPED'].value_counts().sum()*.7

In [0]:
df = pd.read_csv("project_profit.csv", encoding = "ISO-8859-1")
print(df.shape)
drop_list = []
therold = 0.7
for (col_name, col) in df.iteritems():
  is_valid = not df[col_name].value_counts().max() > df[col_name].value_counts().sum()*therold
  if not is_valid:
    drop_list.append(col_name)
    print(col_name)

df = df.drop(columns = drop_list)
print(df.shape)

In [0]:
df = pd.read_csv("project_profit.csv", encoding = "ISO-8859-1")
df.head()

,ProjectFamily,NonIndiaHrsPED,LaborEffortPED,TotalHoursPED,PrincipalProjectCountPED,PMProjectCountPED,LaborHrsPED,EngUniqueCount,EngProjectCountPED,EngHoursPED,EmpCountPED,TypeCode,ProjectManager,ProjectEngineer,Principal,ParentClientTypeCode,ParentClientCode,OwningOrgCode,DivisionHead,ClientTypeCode,ClientState,ClientCode,ClientCity,HoursPTD,TotalBudgetHours,LaborCostMTD,OverheadMTD,LaborOverheadMTD,EffortMTD,CostMTD,BillingMTD,FamilyOverheadMTD,FamilyCostMTD,FamilyRevenueMTD,total profit,profit_per,profit
0,1204001,31.50,2548.52,31.50,0,11,31.50,0,11,0.00,16,P06,121014,121014.0,121012,DE,UPTOWN,Z91,121012,DE,Texas,UPTOWN,Houston,0.0,0.0,910.19,0.000000,0.000000,2603.65,960.31,2603.63,0.000000,960.31,2548.52,1588.210000,0.623189,1
1,2300002,770.75,45814.18,770.75,1196,382,770.75,91,653,582.25,381,P06,230173,230173.0,230173,DE,LOUETT,T03,230009,DE,Texas,LOUETT,HOUSTON,0.0,55.0,9766.85,19748.315675,19748.315675,29553.13,11611.41,56857.39,40641.283837,23287.80,74610.86,10681.776163,0.143167,1
2,2300028,9.00,607.49,9.00,141,71,9.00,1,71,3.00,5,E02,230018,230018.0,230021,AR,KENN,C03,230009,AR,Missouri,KENN,ST. LOUIS,0.0,86.0,216.96,433.703040,433.703040,607.49,216.96,6000.00,433.703040,216.96,6000.00,5349.336960,0.891556,1
3,2300033,19.00,1173.18,19.00,599,87,19.00,15,385,13.00,39,P06,230195,230141.0,230021,AR,ZCA,C03,230009,AR,Texas,ZCA,Houston,0.0,0.0,458.40,930.186018,930.186018,1386.12,557.70,235.45,5348.315848,2944.55,18870.44,10577.574152,0.560537,1
4,2300036,11.50,907.59,11.50,269,41,11.50,6,112,11.50,14,C06,230125,230018.0,230021,AR,J*R,C03,230009,AR,Texas,J*R,HOUSTON,2.5,70.0,302.53,573.544123,573.544123,933.65,328.59,1211.30,573.544123,328.59,1200.00,297.865877,0.248222,1


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import scipy
import csv
import time
import sys
le = LabelEncoder() 
np.set_printoptions(threshold=100)

df = pd.read_csv("project_profit.csv", encoding = "ISO-8859-1")
rf = RandomForestClassifier(n_estimators=100)
df = df.drop(columns=['ProjectFamily'])

catagory_idx = []
i = 0
for (col_name, col) in df.iteritems():
  if col.dtype == 'object':
    df[col_name] = le.fit_transform(df[col_name])
    catagory_idx.append(i)
  i += 1
print(df.columns[catagory_idx])

#print(catagory_idx)
#print(df.columns[catagory_idx])
onehotencoder = OneHotEncoder(categorical_features = catagory_idx)

#print(df)
#df['profit_per'][(df['profit_per']==float('-inf'))] = -sys.maxsize - 1
#df['profit_per'][(df['profit_per']==float('inf'))] = sys.maxsize
df = df.dropna(axis=0)
response = df['profit']
# df = df.drop(columns=['profit', 'profit_per', 'total profit', 'FamilyRevenueMTD', 'FamilyOverheadMTD', 'FamilyCostMTD', 'CostMTD', 'OverheadMTD'])
features = list(df.columns)

# print(df.shape)
# print(response.shape)

print(response.shape)
idx = np.random.permutation(response.shape[0])
print(idx.shape)

# print((df == np.nan).sum())
rand_pred = np.array(df)
#rand_pred = onehotencoder.fit_transform(df)
#rand_pred = rand_pred.toarray()
#print(scipy.sparse.issparse(rand_pred))
rand_resp = np.array(response)[idx]

rand_pred = rand_pred[idx]
#print(rand_pred)
# print(rand_pred[10])
valid_pct = 0.2
# print(rand_pred.shape, rand_resp.shape)
valid_len = (int)(valid_pct*response.shape[0])
X_valid, Y_valid = rand_pred[idx[:valid_len]], rand_resp[idx[:valid_len]]
X_train, Y_train = rand_pred[idx[valid_len:]], rand_resp[idx[valid_len:]]
#print(set(idx[:valid_len]).intersection(set(idx[valid_len:])))

# print(Y_valid.sum(), Y_valid.count())
# print(Y_train.sum(), Y_train.count())
#rf.fit(X_train, Y_train)
#print((np.inf == Y_train).sum())
#print(X_train.shape, Y_train.shape)
'''
for y in Y_train:
  if y != 0 and y != 1:
    print(y)
'''
'''
print(X_train.shape, Y_train.shape)
print(Y_train.sum()/Y_train.shape[0])
print(X_valid.shape, Y_valid.shape)
print(Y_valid.sum()/Y_valid.shape[0])
'''
res = rf.fit(X_train, Y_train)

print(sum(res.predict(X_valid) == Y_valid)/len(Y_valid))

print(sum(res.predict(X_train) == Y_train)/len(Y_train))


importances = rf.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(20,10))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color = 'r', align = 'center')
print(features)
plt.yticks(range(len(indices)), features)
plt.xlabel("Relative Importance")
plt.show()



Index(['TypeCode', 'ParentClientTypeCode', 'ParentClientCode', 'OwningOrgCode',
       'ClientTypeCode', 'ClientState', 'ClientCode', 'ClientCity'],
      dtype='object')
(10201,)
(10201,)


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: ignored

Index(['TypeCode', 'ParentClientTypeCode', 'ParentClientCode', 'OwningOrgCode',
       'ClientTypeCode', 'ClientState', 'ClientCode', 'ClientCity'],
      dtype='object')
(10201,)
(10201,)


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: ignored